# Functions & Closures

In [2]:
#[derive(Debug, PartialEq, Clone)]
struct Person {
    name: String,
    age: u8,
    height: f32
}

fn create_people() -> Vec<Person> {
    vec![
        Person {
            name: "Alice".to_string(),
            age: 20,
            height: 1.7
        },
        Person {
            name: "Bob".to_string(),
            age: 25,
            height: 1.8
        },
        Person {
            name: "Charlie".to_string(),
            age: 30,
            height: 1.9
        },
        Person {
            name: "David".to_string(),
            age: 35,
            height: 1.75
        },
        Person {
            name: "Eve".to_string(),
            age: 40,
            height: 1.75
        },
        Person {
            name: "Adam".to_string(),
            age: 16,
            height: 1.6
        },
        Person {
            name: "Marry".to_string(),
            age: 11,
            height: 1.5
        }
    ]
}

let people = create_people();

let people_sorted_by_age = {
    let mut people = people.clone();
    people.sort_by_key(|person| person.age);
    people
};

let people_sorted_by_height = {
    let mut people = people.clone();
    people.sort_by(|person, other|  f32::total_cmp(&person.height, &other.height));
    people
};

println!("People ordered by age: {:?}", people_sorted_by_age);
println!("People ordered by height: {:?}", people_sorted_by_height);

People ordered by age: [Person { name: "Marry", age: 11, height: 1.5 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }]
People ordered by height: [Person { name: "Marry", age: 11, height: 1.5 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }]


## Closures Borrow by Default

In [3]:
use std::collections::HashMap;

fn create_name_translator() -> HashMap<String, String> {
    [
        ("Alice".to_string(), "Alicja".to_string()),
        ("Bob".to_string(), "Robert".to_string()),
        ("Charlie".to_string(), "Karol".to_string()),
        ("David".to_string(), "Dawid".to_string()),
        ("Eve".to_string(), "Ewa".to_string()),
        ("Adam".to_string(), "Adam".to_string()),
        ("Marry".to_string(), "Maria".to_string())
    ].iter().cloned().collect()
}

fn main() {
    let people = create_people();

    let name_translator: HashMap<_, _> = create_name_translator();

    let person_name_translator = |person: &Person| {
        let translated_name = name_translator.get(&person.name).unwrap_or(&person.name); // name_translator is borrowed
        Person {
            name: translated_name.to_string(),
            age: person.age,
            height: person.height
        }
    };

    let people_translated: Vec<Person> = people.iter().map(person_name_translator).collect();

    println!("People with translated names: {:?}", people_translated);
}

main();

People with translated names: [Person { name: "Alicja", age: 20, height: 1.7 }, Person { name: "Robert", age: 25, height: 1.8 }, Person { name: "Karol", age: 30, height: 1.9 }, Person { name: "Dawid", age: 35, height: 1.75 }, Person { name: "Ewa", age: 40, height: 1.75 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Maria", age: 11, height: 1.5 }]


## Closures That Move

In [7]:
fn main() {
    let people: Vec<Person> = create_people();
    let name_translator: HashMap<String, String> = create_name_translator();

    let person_name_translator = move |person: &Person| {
        let translated_name = name_translator.get(&person.name).unwrap_or(&person.name); // name_translator is moved
        Person {
            name: translated_name.to_string(),
            age: person.age,
            height: person.height
        }
    };

    let thd = std::thread::spawn(move || {    
        let people_translated: Vec<Person> = people.iter().map(person_name_translator).collect(); // people & person_name_translator are moved
    
        println!("People with translated names: {:?}", people_translated);
    });

    thd.join().unwrap();
}

main();

People with translated names: [Person { name: "Alicja", age: 20, height: 1.7 }, Person { name: "Robert", age: 25, height: 1.8 }, Person { name: "Karol", age: 30, height: 1.9 }, Person { name: "Dawid", age: 35, height: 1.75 }, Person { name: "Ewa", age: 40, height: 1.75 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Maria", age: 11, height: 1.5 }]


## Function & Closure Types

### Function Type

In [8]:
fn is_adult(person: &Person) -> bool {
    person.age >= 18
}

The function `is_adult` has a type: `fn(&Person) -> bool`

In [9]:
let f_predicate: fn(&Person) -> bool = is_adult;

fn filter_by_predicate(people: &Vec<Person>, predicate: fn(&Person) -> bool) -> Vec<Person> {
    let mut qualified: Vec<Person> = Vec::new();
    for person in people {
        if predicate(person) {
            qualified.push(person.clone());
        }
    }
    qualified
}

let adults = filter_by_predicate(&people, is_adult);

println!("Adults: {:?}", adults);

Adults: [Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }]


When function expects a function as an argument, it accepts a closure that captures nothing:

In [10]:
let tall_people = filter_by_predicate(&people, |person| person.height > 1.75);

println!("Tall people: {:?}", tall_people);

Tall people: [Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }]


A closure that captures variables from the enclosing scope cannot be passed to a function that accepts a function type:

In [11]:
let height_limit = 1.75;
let tall_people = filter_by_predicate(&people, |person| person.height > height_limit);

Error: mismatched types

### `Fn` Trait

* Functions and closures implement the `Fn` trait, which is the trait of things that can be called like functions.

In [12]:
fn filter_by_predicate<F>(people: &Vec<Person>, predicate: F) -> Vec<Person>
    where F: Fn(&Person) -> bool 
{
    let mut qualified: Vec<Person> = Vec::new();
    for person in people {
        if predicate(person) {
            qualified.push(person.clone());
        }
    }
    qualified
}

In [13]:
let adults = filter_by_predicate(&people, is_adult); // using function pointer
println!("Adults: {:?}", adults);

let height_limit = 1.75;
let tall_people = filter_by_predicate(&people, |person| person.height > height_limit); // using closure that captures (borrows) height_limit
println!("Tall people: {:?}", tall_people);

Adults: [Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }]
Tall people: [Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }]


* Now the function `filter_by_predicate` is generic. It accepts any function or closure `F` that implements the special trait `Fn(&Person) -> bool`.
* The trait `Fn(&Person) -> bool` automatically implemented by all functions and closures that can be called with a `&Person` argument and return a `bool`.

### `FnMut` Trait

* The `FnMut` trait is a subtrait of `Fn`. It allows the closure to mutate the captured variables.

In [14]:
fn create_generator(seed: i32) -> impl FnMut() -> i32 {
    let mut state = seed;
    move || {
        state += 1;  // mutating captured state
        state
    }
}

fn main() {
    let mut generator = create_generator(10);
    println!("Generated number: {}", generator());
    println!("Generated number: {}", generator());
    println!("Generated number: {}", generator());
}

main();

Generated number: 11
Generated number: 12
Generated number: 13


In [14]:
fn main() {
    let mut state = 10;

    let mut f = || {
        state += 1; // mutating captured state
        state
    };

    let mut boxed_f: Box<dyn FnMut()->i32> = Box::new(f);

    boxed_f();
    boxed_f();
}

main();

### `FnOnce` Trait

* Some functions and closures can be called only once. They are not allowed to be called multiple times.
* The trait `FnOnce` is the trait of things that can be called once. It is implemented by functions and closures that can be called only once.
  * The `FnOnce` trait is a subtrait of the `FnMut` trait, which is a subtrait of the `Fn` trait.

In [ ]:
fn main() {
    let my_string = "Hello, World!".to_string();
    let f = || drop(my_string); // this closure should be called only once

    f();
    f();
}

In [18]:
fn main() {
    let mut data = vec![1, 2, 3, 4, 5];

    let mut f_once = || {
        data.push(665); // mutating captured data
        data // moving the data out of the closure
    };

    let mut f_once_boxed: Box<dyn FnOnce() -> Vec<i32>> = Box::new(f_once);

    let target_data = f_once_boxed();
    println!("Data: {:?}", target_data);
}

main();

Data: [1, 2, 3, 4, 5, 665]


* `Fn` is the family of closures and functions that you can call multiple times without restriction. This highest category also includes all fn functions.

* `FnMut` is the family of closures that can be called multiple times if the closure itself is declared mut.

* `FnOnce` is the family of closures that can be called once, if the caller owns the closure.

* Every `Fn` meets the requirements for `FnMut`, and every `FnMut` meets the requirements for `FnOnce`

In [61]:
// mod explain {
//     // Pseudocode for `Fn`, `FnMut`, and `FnOnce` traits.
//     trait Fn() -> R {
//         fn call(&self) -> R;
//     }

//     trait FnMut() -> R {
//         fn call_mut(&mut self) -> R;
//     }

//     trait FnOnce() -> R {
//         fn call_once(self) -> R;
//     }
// }

## Copy and Clone for Closures

* Just as Rust automatically figures out which closures can be called only once, it can figure out which closures can implement `Copy` and `Clone`, and which cannot.

In [19]:
fn main() {
    let y = 4;
    let add_y = move |x| x + y;
    let copy_of_add_y = add_y; // closure is copied
    assert_eq!(add_y(copy_of_add_y(3)), 11);

    let word: String = "Hello".to_string();
    let append_word = move |mut s: String| {
        s.push_str(&word); 
        s
    };
    let copy_of_append_word = append_word.clone(); // closure is cloned
    assert_eq!(append_word(copy_of_append_word(" World".to_string())), " WorldHelloHello");
}

main();

# Callbacks

* Callbacks are functions that are passed as arguments to other functions.
* Callbacks are used to implement event handlers, asynchronous programming, and other patterns.

In [20]:
type Callback = Box<dyn Fn(f32)>;

struct TemperaturMonitor {
    current_temp: f32,
    on_temperature_changed: Option<Callback>,
}

In [21]:
impl TemperaturMonitor {
    fn new() -> Self {
        Self {
            current_temp: 0.0,
            on_temperature_changed: None,
        }
    }

    fn set_on_temperature_changed<F: Fn(f32) + 'static>(&mut self, callback: F) {
        self.on_temperature_changed = Some(Box::new(callback));
    }

    fn set_current_temp(&mut self, temp: f32) {
        self.current_temp = temp;
        if let Some(ref on_temperature_changed) = self.on_temperature_changed {
            on_temperature_changed(self.current_temp);
        }
    }    
}

* Rust makes us add this `'static bound`. Without it, the call to `Box::new(callback)` would be an error, because it’s not safe to store a closure if it contains borrowed references to variables that are about to go out of scope.)

In [22]:
fn main() {
    let mut monitor = TemperaturMonitor::new();
    monitor.set_on_temperature_changed(|temp| println!("Temperature changed to: {}", temp));
    monitor.set_current_temp(22.5);
    monitor.set_current_temp(23.5);

    monitor.set_on_temperature_changed(|temp| println!("Current temperature is: {}°C", temp));
    monitor.set_current_temp(24.7);
    monitor.set_current_temp(27.1);
}

main();

Temperature changed to: 22.5
Temperature changed to: 23.5
Current temperature is: 24.7°C
Current temperature is: 27.1°C
